In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist  # 从keras中加载MNIST数据集

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 预处理：将图像数据归一化到 [0, 1] 范围内，并将标签进行 one-hot 编码
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)  # 将标签转换为one-hot编码
y_test = tf.keras.utils.to_categorical(y_test, 10)

learning_rate = 1e-4  # 学习率
keep_prob_rate = 0.7  # Dropout 保留的概率
max_epoch = 2000  # 最大训练轮数

# 计算模型准确率的函数
def compute_accuracy(v_xs, v_ys):
    global prediction
    v_xs = v_xs[..., tf.newaxis]  # 添加通道维度
    y_pre = model(v_xs, training=False)  # 获取预测结果
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))  # 判断预测是否正确
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))  # 计算准确率
    return accuracy

# 权重初始化函数
def weight_variable(shape):
    initial = tf.random.truncated_normal(shape, stddev=0.1)  # 使用截断正态分布初始化权重
    return tf.Variable(initial)

# 偏置初始化函数
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)  # 偏置初始化为0.1
    return tf.Variable(initial)

# 卷积层的函数
def conv2d(x, W):
    # 使用tf.nn.conv2d进行卷积操作，步幅为1，padding方式为'SAME'
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# 2x2的最大池化操作
def max_pool_2x2(x):
    # 使用tf.nn.max_pool进行2x2的最大池化操作，步幅为2，padding方式为'SAME'
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 使用 Keras 定义模型
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, kernel_size=(7, 7), activation='relu', padding='same')
        self.pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same')
        self.conv2 = tf.keras.layers.Conv2D(64, kernel_size=(5, 5), activation='relu', padding='same')
        self.pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same')
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dropout = tf.keras.layers.Dropout(keep_prob_rate)
        self.fc2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x, training=training)
        return self.fc2(x)

# 创建模型实例
model = CNN()

# 定义损失函数
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

# 定义优化器
optimizer = tf.keras.optimizers.Adam(learning_rate)

# 训练步骤
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        predictions = model(x_batch, training=True)
        loss = loss_fn(y_batch, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# 启动训练过程
for epoch in range(max_epoch):
    for i in range(0, len(x_train), 100):
        batch_xs = x_train[i:i+100]
        batch_ys = y_train[i:i+100]
        batch_xs = batch_xs[..., tf.newaxis]  # 添加通道维度
        batch_xs = tf.cast(batch_xs, tf.float32)  # 转换数据类型

        # 进行一次训练步骤
        loss = train_step(batch_xs, batch_ys)

    if epoch % 100 == 0:
        accuracy = compute_accuracy(x_test, y_test)
        print(f"Epoch {epoch}, Test Accuracy: {accuracy.numpy():.4f}")


Epoch 0, Test Accuracy: 0.9617
